In [ ]:
#[[26,17, 27]]: left brow, right brow, upper nose (between eyes)

corner_points = [24, 19, 68]
corner_points = -landmarks1[:,0][corner_points], -landmarks1[:,1][corner_points]

# OVERLAY_POINTS = [
#     LEFT_EYE_POINTS + RIGHT_EYE_POINTS + #(LEFT_BROW_POINTS+[68]) 
#     #+ (RIGHT_BROW_POINTS+[68]),
#     (NOSE_POINTS) + MOUTH_POINTS + 
#     [24, 19, 68]
# ]


plt.scatter(-landmarks1[:,0], -landmarks1[:,1], alpha=0.5)
plt.scatter(corner_points[0], corner_points[1], color='red')

In [ ]:
# plt.scatter(-np.where(mask>0.)[1], -np.where(mask>0.)[0])
# plt.show()

im = numpy.zeros(im1.shape[:2], dtype=numpy.float64)

for group in OVERLAY_POINTS:

    points = landmarks1[group]
    points = cv2.convexHull(points)
    cv2.fillConvexPoly(im, points, color=1)
    
group = [[24, 19, 68]]

points = landmarks1[group]
points = cv2.convexHull(points)
cv2.fillConvexPoly(im, points, color=2)
    
    
im = numpy.array([im, im, im]).transpose((1, 2, 0))
im = (cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0) > 0) * 1.0
im = cv2.GaussianBlur(im, (FEATHER_AMOUNT, FEATHER_AMOUNT), 0)

plt.imshow(im)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(annotate_landmarks(im1, landmarks1))
plt.imshow(warped_mask, alpha=0.5)

In [ ]:
# display(Image(filename='ava.jpg', width=200, height=100), )
# display(Image(filename='porn.jpg', width=200, height=100), )
# display(Image(filename='output.jpg', width=200, height=100), ) 

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(output_im)
# plt.imshow(warped_mask, alpha=0.5)